<a href="https://colab.research.google.com/github/JacePalmer/Machine-Learning/blob/main/W3_BMEG4983.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src=https://brand.uark.edu/_resources/images/UA_Logo_Horizontal.jpg width="400" height="96">

####BMEG 4983 - Genome Engineering and Synthetic Biology -
####For more information, check out the Nelson lab for Therapeutic Genome Engineering (https://nelsonlab.uark.edu/)

#3. Data Workshop 3 - CRISPR gRNA design
Update for 2025


The goal for today is to take a sequence file and design CRISPR gRNAs that tile the whole sequence (Every available PAM). This will combine our previous two lessons in using python, data I/O, and handling sequence data.

# 3.1 Outline


*   Data Input - We need to get sequence data into the computer
*   Dealing with sequence data
*   Iterating through a data set
*   Data output


# 3.2 Data Input

First we need to get some data. There are several ways to do this.
1. Import from your computer
2. Import from online
3. Import from google drive

The file type we are using is a fasta file
It has a header defined by '>'
The remaining lines are sequence lines.
Fasta files may have multiple entries so you may have to write a function that can deal with multiple fasta sequences

A few notes / reminders
*   I tried out a shortcut script to get genome data from USC genome browswer (warning - this isn't great)
*   You can pull data from NIH (be careful doing this!) You can read tutorials online but don't get banned
*   Data from github - this is the easiest for this class and so I will focus the code below on that method

For more information on data input and output, see this document:
https://colab.research.google.com/notebooks/io.ipynb



## 3.2.1 Getting coordinates for your gene by exon

## 3.2.1 Pulling data from genome browser

One way I found is to scrape xml data from the genome browser

You can get sequence data in XML format from here:
http://genome.ucsc.edu/cgi-bin/das/hg38/dna?segment=chrX:83803113,83803313

You can see above that this uses hg38 (human genome version 38) and pulls data from the X chromosome, coordinates between 83803113 to 83803313. Make sure not to grab a large section.


In [ ]:
#This could be better

import pandas as pd
df = pd.read_csv('http://genome.ucsc.edu/cgi-bin/das/hg38/dna?segment=chrX:83803113,83803313')
seq = ''
index = 4
print(df)
while True:
  df.iat[index,0]
  seq = seq+df.iat[index,0]
  index = index +1
  if (df.iat[index,0]=="</DNA>"):
    print('break')
    break
print(seq)

                <?xml version="1.0" standalone="no"?>
0   <!DOCTYPE DASDNA SYSTEM "http://www.biodas.org...
1                                            <DASDNA>
2   <SEQUENCE id="chrX" start="83803113" stop="838...
3                                  <DNA length="201">
4   tcgatttaatcattttgtgtactggtaggctttgattcctttctgt...
5   cctgtgtatctactgcaggatttttagtgacattgcaatgaggctt...
6   acagcatcctatagtgatatcagtttattttaagttgataacaact...
7   tttgacatgtctttattttctgttagtaccattataatacatatat...
8                                                   a
9                                              </DNA>
10                                        </SEQUENCE>
11                                          </DASDNA>
break
tcgatttaatcattttgtgtactggtaggctttgattcctttctgtttctcctgtgtatctactgcaggatttttagtgacattgcaatgaggcttacatacagcatcctatagtgatatcagtttattttaagttgataacaacttaattttgacatgtctttattttctgttagtaccattataatacatatattgcaa


## 3.2.2 Pulling data from github
One way to get data into google colab is to pull directly from github. For you to do this with your own sequence, you will need to set up a github account.

In [ ]:
#Function to grab a fasta from github
import pandas as pd
def get_github_fasta(url):

  df = pd.read_csv('https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/test_seq.fa');
  header_list = df.columns.values.tolist()
  header = header_list[0]

  seq = ''
  index = 0
  for i in range(0, len(df)):
    seq = seq+df.iat[i,0]

  return header, seq
header, seq = get_github_fasta('https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/test_seq.fa')
print(header)
print(seq)

>Portion of DMD gene
tcgatttaatcattttgtgtactggtaggctttgattcctttctgtttctcctgtgtatctactgcaggatttttagtgacattgcaatgaggcttacatacagcatcctatagtgatatcagtttattttaagttgataacaacttaattttgacatgtctttattttctgttagtaccattataatacatatattgcaa


## 3.2.3 Getting data from an input file
We could also upload a file from our own PC:

```python
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
```

We will need to write a quick wrapper function to get this fasta data and save a header and sequence data:

In [ ]:
#FYI I'm having trouble getting this to work all of the time. It seems browser dependent. It may work for you
#UPDATE it looks like a third party cookie issue. You can google the following if you have trouble and follow
#the directions: Upload widget is only available when the cell has been executed in the current browser session. Please rerun this cell to enable.

from google.colab import files
def get_fasta_from_pc():
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  seq_list=uploaded[fn].decode("utf-8").splitlines()
  header = seq_list[0]
  header = header.replace('>','')

  seq = ''
  for i in range(1,len(seq_list)):
    seq = seq+seq_list[i]

  return header, seq
#Now get the sequence data out of the file
my_header, my_seq = get_fasta_from_pc()
print('header')
print(my_header)
print('my seq')
print(my_seq)

#consider what would happen if you had more than one entry in the fasta file


Saving TP53.fasta to TP53 (5).fasta
User uploaded file "TP53 (5).fasta" with length 19534 bytes
hg38_dna range=chr17:7668421-7687490 5'pad=0 3'pad=0 strand=+ repeatMasking=none
did it work?
TGGCAGCAAAGTTTTATTGTAAAATAAGAGATCGATATAAAAATGGGATATAAAAAGGGAGAAGGAGGGGAAGGGTGGGGTGAAAATGCAGATGTGCTTGCAGAATGTAAAAGATGTTGACCCTTCCAGCTGGACGTGGTGGCTCACAATTGTAATCCCAGCACTCTGGGAGGCTGAGACAGGTGGATCGCCTGAGCCCAGGAGTTTGAGACCAGCCTGGGCAACACTGTGAGACCCCATCTCTACAAAACATGCAAAAGTTGGCTGGCCATGGTGGCATGAACCTGTGGTCCCAGCTACTCCGGAGGCTGAGGCAGGACTGCTCGAGCCGGGGAGGCAAAGGCTGCAGTAAGCCAAGATCACGCCACTCCACTCCAGCCTGGGCAACAAAGCGAGACCCAGTCTCAAAGAAAAAGAAAAAAAAAAAAAAAAAAGAAAAAAGAAATTGACCCTGAGCATAAAACAAGTCTTGGTGGATCCAGATCATCATATACAAGAGATGAAATCCTCCAGGGTGTGGGATGGGGTGAGATTTCCTTTTAGGTACTAAGGTTCACCAAGAGGTTGTCAGACAGGGTTTGGCTGGGCCAGCAGAGACTTGACAACTCCCTCTACCTAACCAGCTGCCCAACTGTAGAAACTACCAACCCACCGACCAACAGGGAGAGGGAACAAGCACCCTCAAGGGGGTCAAGTTCTAGACCCCATGTAATAAAAGGTGGTTTCAAGGCCAGATGTACATTATTTCATTAACCCTCACAATGCACTCTGTGAGGTAGGTGCAAATGCCAGCATTTCACAGATATGGGC

### 3.2.4 Getting data from Table Browser

**First find your gene in genome browser**


<img src=https://raw.githubusercontent.com/chrisnelsonlab/BMEG49803_2025/refs/heads/main/TB1.png>


**Now go to table browser with your information:**
<img src=https://raw.githubusercontent.com/chrisnelsonlab/BMEG49803_2025/refs/heads/main/TB2.png>


**Choose paste list and put in your target gene and submit**
<img src=https://raw.githubusercontent.com/chrisnelsonlab/BMEG49803_2025/refs/heads/main/TB3.png>

**Choose output as sequence**
<img src=https://raw.githubusercontent.com/chrisnelsonlab/BMEG49803_2025/refs/heads/main/TB4.png>


**Choose genomic**
<img src=https://raw.githubusercontent.com/chrisnelsonlab/BMEG49803_2025/refs/heads/main/TB5.png>

**Now change some of these to get the output you want. I would deselect introns**

<img src=https://raw.githubusercontent.com/chrisnelsonlab/BMEG49803_2025/refs/heads/main/TB6.png>

**Finally here is the output** Can you import this and get the data into a usable format in python?

If you manage to write a script to talk with UCSC or NIH directly to do this, let me know.

# 3.3 Dealing with sequence data


*   Reverse complement function
*   Function to check if a sequence is a PAM. Compares two sequence motifs
  Can you get this to work with the ambigious nucleotide letters?



## 3.3.1 Reverse complement

We will need to find CRISPR gRNAs on both strands. To do this, we will need to reverse complement the sequence. Let's write our own sequence:

To reverse compelement, we need to:
1. reverse the string (remember the -1 trick!)
2. Compelement the string (remember the intermediate trick)
3. Extra challenge, can you handle all of the ambiguous letters?

Reminder:
* A, T, C, G – already defined
* N = “Any” = A,T,C, or G
* R = “puRine” = G or A
* Y = “pYrimidine” = C or T
* S = “Strong” = G or C
* W = “Weak” = A or T
* K = “Keto” = G or T
* M = “Amino” = A or C
* B = “anything but A” = T, C, or G
* D = “anything but C” = A, T, or G
* H = “Anything but G” = A,T, or C
* V = “anything but T” = A, C, or G


In [ ]:
def reverse_complement(my_seq):
 #Make a dictionary with ambiguous symbols
  complements = { "A":"T", "T":"A","C":"G","G":"C","N":"N","R":"Y","Y":"R",
                 "S":"S","W":"W","K":"M","M":"K","B":"V","V":"B","D":"H","H":"D",
                 "a":"t", "t":"a","c":"g","g":"c","n":"n","r":"y","y":"r",
                 "s":"s","w":"w","k":"m","m":"k","b":"v","v":"b","d":"h","h":"d"}
  #This line is doing the work. It is iterating through a reversed my_seq
  #and putting each nucleotide into the variable base. For each of those
  #it looks up in the dictionary what the complement is and adds those to
  #a growing string.
  return "".join(complements[base] for base in reversed(my_seq))

#Example
print(reverse_complement('ATGCTGANNNnnnRRGatcg'))

cgatCYYnnnNNNTCAGCAT


In [ ]:
#@title Hidden Reverse Complement Function. Double click to reveal
def reverse_complement(seq):
  #This function returns a reverse complement of a sequence. Not designed
  #to be computationally efficient
  #Also only handles ATCG
  seq = seq.upper()
  seq = seq[::-1]
  seq = seq.replace("A", "t")
  seq = seq.replace("C", "g")
  seq = seq.replace("T", "a")
  seq = seq.replace("G", "c")
  seq = seq.upper()
  return seq
print(reverse_complement('ATCG'))

CGAT


## 3.3.2 Finding PAM sequences
The Cas9 protein seeks out pam sequences. In SpCas9, it seeks 5'-NGG-3'

We need to make a function that decides if a sequence is a PAM. There are a fewways we could do this:
1. Iterate through our sequence, take three characters out of the string, and see if that is an NGG. We know that N = anything, so we really only need to look at the last two:
```python
if(my_sequence[i+1:i+3] = 'GG'):
  #record the previous 20 bases
else:
  # not a pam, we really don't need this else
```
2. Another way is to be more flexible about the PAM - e.g. let the user define the PAM. We would need to iterate through the sequence and check if each is a PAM. Could this also handle ambiguous letters?

Try writing a function below that compares two sequences to see if the sequene is the same. Try to make the function as flexible as possible (different lengths, ambiguous letters, etc).

I have my version hidden below. It returns true if the sequence matches, false if it doesn't


In [ ]:
#Function takes in two input arguments, a sequence to compare and a PAM
def is_pam(seq,PAM):
  #Your code here
  return True #or false?


Hidden Code below for is_pam function

In [ ]:
def is_pam(seq,PAM):
    #This function decides if a given seq is a PAM. This should work for any
    #length of PAM. This takes into account generic nucleotide letters
    for x,char in enumerate(PAM):
        if char =='N':
            continue
        elif char =="A" and seq[x]=="A":
            continue
        elif char =="T" and seq[x]=="T":
            continue
        elif char =="C" and seq[x]=="C":
            continue
        elif char =="G" and seq[x]=="G":
            continue
        elif char =="R" and (seq[x]=="G" or seq[x]=="A"):
            continue
        elif char =="Y" and (seq[x]=="C" or seq[x]=="T"):
            continue
        elif char =="S" and (seq[x]=="G" or seq[x]=="C"):
            continue
        elif char =="W" and (seq[x]=="A" or seq[x]=="T"):
            continue
        elif char =="K" and (seq[x]=="G" or seq[x]=="T"):
            continue
        elif char =="M" and (seq[x]=="A" or seq[x]=="C"):
            continue
        elif char =="B" and (seq[x]=="C" or seq[x]=="G" or seq[x]=="T"):
            continue
        elif char =="D" and (seq[x]=="A" or seq[x]=="G" or seq[x]=="T"):
            continue
        elif char =="H" and (seq[x]=="A" or seq[x]=="C" or seq[x]=="T"):
            continue
        elif char =="V" and (seq[x]=="A" or seq[x]=="C" or seq[x]=="G"):
            continue
        else:
            return False
    return True
print(is_pam('ATC','NGG')) # Should be false
print(is_pam('ATGAAT', 'NNGRRT')) #Should be true

False
True


## 3.3.3 Iteratating through sequence data

Next we need to iterate through the sequence data and find all of the protospacer sequences.

We need to search both strands.

First, we can search the sequence that we were provided. To avoid out-of-bounds errors, there is no reason to start until 20bp into the sequence

Next, we will reverse complement the sequence and search the reverse complmenet strand.

We can add these to a list (easy). More difficult, can you make a better table (or use pandas) to make a table that records the sequence, PAM, positition, and strand? Something like:

| Position      | Protospacer | PAM | Strand |
| ----------- | ----------- | ----------- | ----------- |
| 200      | GTATCTGATCGTACTGTTATA       | TGG       | +       |
| 235      | CATCTGGTACTTGTCGTTATT       | AGG       | -       |






In [ ]:
def get_protospacer_list():
  #Try to make a function that does this. You can use the function
  #below as inspiration if you get stuck.

In [ ]:
import pandas as pd
#This function takes in a desired protospacer size, a PAM, and a sequence
#It needs the functions is_pam, and reverse_complement to work
def get_protospacer_list(protospacer_size,PAM,seq):
  protospacer_size = 20
  PAM = 'NGG'
  Cut_Position_from_PAM = 3
  df = pd.DataFrame(columns = ['CutPosition', 'Protospacer','PAM', 'Strand'])

  # Create the pandas DataFrame
  try:
    print('your sequence:')
    print('length: '+str(len(seq))+' bp')
    print(seq)
  except:
    print('you need to define a sequence first')
    print('for now, I will use a default sequence')
    seq = 'tcgatttaatcattttgtgtactggtaggctttgattcctttctgtttctcctgtgtatctactgcaggatttttagtgacattgcaatgaggcttacatacagcatcctatagtgatatcagtttattttaagttgataacaacttaattttgacatgtctttattttctgttagtaccattataatacatatattgcaa'

  seq=seq.upper()

  for i in range(protospacer_size,len(seq)-len(PAM)):
    if(is_pam(seq[i:i+len(PAM)],PAM)):
    #troubleshooting
    #print(i)
    #print(seq[i:i+len(PAM)])
    #print(seq[i-protospacer_size:i])
      df = df.append({'CutPosition':i-Cut_Position_from_PAM,'Protospacer':seq[i-protospacer_size:i],'PAM':seq[i:i+len(PAM)],'Strand':'+'}, ignore_index=True)

  #Check reverse complement
  RCseq = reverse_complement(seq)
  fasize = len(seq)
  for i in range(protospacer_size,len(RCseq)-len(PAM)):
    if(is_pam(RCseq[i:i+len(PAM)],PAM)):
      #troubleshooting
      #print(i)
      #print(RCseq[i:i+len(PAM)])
      #print(RCseq[i-protospacer_size:i])

      #Notice the change in position to make sure we have the original strands position
      df = df.append({'CutPosition':str(fasize-i+Cut_Position_from_PAM),'Protospacer':RCseq[i-protospacer_size:i],'PAM':RCseq[i:i+len(PAM)],'Strand':'-'}, ignore_index=True)
  return df

my_seq = 'tcgatttaatcattttgtgtactggtaggctttgattcctttctgtttctcctgtgtatctactgcaggatttttagtgacattgcaatgaggcttacatacagcatcctatagtgatatcagtttattttaagttgataacaacttaattttgacatgtctttattttctgttagtaccattataatacatatattgcaa'
print(get_protospacer_list(20,'NGG',my_seq))

your sequence:
length: 201 bp
tcgatttaatcattttgtgtactggtaggctttgattcctttctgtttctcctgtgtatctactgcaggatttttagtgacattgcaatgaggcttacatacagcatcctatagtgatatcagtttattttaagttgataacaacttaattttgacatgtctttattttctgttagtaccattataatacatatattgcaa
  CutPosition           Protospacer  PAM Strand
0          19  GATTTAATCATTTTGTGTAC  TGG      +
1          23  TAATCATTTTGTGTACTGGT  AGG      +
2          63  TTCTCCTGTGTATCTACTGC  AGG      +
3          87  TTTTTAGTGACATTGCAATG  AGG      +
4         184  TTGCAATATATGTATTATAA  TGG      -
5         113  AAATAAACTGATATCACTAT  AGG      -
6          56  AAATCCTGCAGTAGATACAC  AGG      -
7          43  GATACACAGGAGAAACAGAA  AGG      -


In [ ]:
Str = "tcgatttaatcattttgtgtactggtaggctttgattcctttc"
print(len(Str))
print(57-43)

43
14


#3.4 Data output and putting it all together
Finally, we need to output the data into something useful for later. We could copy/paste the output from the function, but we could also export the data onto our own computer for future use.

In google colab, you can save data from google colab saved temporarilly over to the left (see the foldericon). Also, you can save to your google drive. You can also prompt a downloading window. We can export our table as a csv

```python
from google.colab import files
df.to_csv('filename.csv')
files.download('filename.csv')
```

In [ ]:
from google.colab import files
#get sequence from github (input section 3.2)
#header, my_seq = get_github_fasta('https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/test_seq.fa')

#ALTERNATE - Get data from your PC - uncomment to try it out
#header, my_seq = get_fasta_from_pc()

#You can also put a sequence in here directly
#replace with imported sequence if your prefer, this sequence is for demo purposes
my_seq = 'AAAGGTTATGCAGCGCGTGAACATGATCATGGCAGAATCACCAGGCCTCATCACCATCTGCCTTTTAGGATATCTACTCAGTGCTGAATGTACAGGTTTGT'
#Dealing with sequence data (Section 3.3)
#You can change the protospacer sizeand PAM here
df = get_protospacer_list(20,'NGG',my_seq)

#Output (Section 3.4)
df.to_csv('my_protospacers.csv')
files.download('my_protospacers.csv')




your sequence:
length: 101 bp
AAAGGTTATGCAGCGCGTGAACATGATCATGGCAGAATCACCAGGCCTCATCACCATCTGCCTTTTAGGATATCTACTCAGTGCTGAATGTACAGGTTTGT


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 3.5 Review

**To recap: We designed for every possible CRISPR gRNA by:**
* importing our own fasta file
* iterating through the sequence
* Finding a PAM
* reverse complementing the whole thing
* Saving data as a table in pandas
* Exporting data as a csv


**There are other improvments we could make:**
* Removing poly_N (polyT) - guide RNAs with poly nucleotides often don't work or are not very specific. Four Ts in a row stops the gRNA from working
* Removing extreme GC content (less than 20% greater than 80%) - Guide RNAs with GC content (the percentage of Gs and Cs) less than 20 or larger than 80 seem to not work as well. Can you make a fucntion to remove those?
* Off-targets - more difficult, can you search the whole genome for mismatches to these gRNAs? This may not be possible in free google colab.

#3.6 HW3 - Designing a library of CRISPR gRNAs

##3.6.1 HW3 Q1 - Get exon coordinates for your gene
Use hg38. Choose just one isoform. Find the starting location and the end location for each exon. Store these in an appropriate data structure of your choice.

In order to determine a more efficient way of doing this, I have opted for using the following curl request within Ubuntu (Linux Operator)

curl -s "https://genome.ucsc.edu/cgi-bin/das/hg38/dna?segment=chr22:44150657,44150740" | grep -v "<"

gctcggatgatcctcagtggtctggggtgcaggcttcaaacctgtagctg
tctagtgacagagtggttcaattccacctttgta

##3.6.2 HW3 Q2 - Find the exon sequences of your gene for targeting with CRISPR gRNAs.
Include 20 bp on each side of the exon to allow for gRNAs to target the genomic DNA near splice sites. Make it a fasta file with an entry for each exon. Lower case can be intron sequence (20 bp flanking) and upper case can be exon sequence.

##3.6.3 HW3 Q3 -
Write (or adapt) a function that scans a sequence for all gRNAs. Make sure to include a check that the cutting location is in the coding region of the gene.

##3.6.3 HW3 Q4 - Design a gRNA libary for your gene
Considerations:
Design a **saturating** gRNA library for the **coding sequence** of your gene. To prevent long genes from causing issues, you can limit the library to 2000 bp of coding sequence or 100 gRNAs, whichever comes first.





#3.7 Sample solution

**Q1 Exon coordinates for TP53**


In [ ]:
#Derived manually from genome browser
#chr17:7,676,521-7,676,622
#chr17:7,676,382-7,676,403
#chr17:7,675,993-7,676,272

**Q2 Exon seqeunces of TP53**

This code will pull 20 bp flanking each side of the exon, put those in lower case and put the coding region in upper case.


In [ ]:
#Here is a hacky version of getting sequence data by chromosome from UCSC
#The function takes in a chromosome and start and stop coords. I made it
#where it will give an error if you ask for more than 1000 bp
import pandas as pd
def getseqdatafromcoords(chr_num_as_str, start, stop):
  if(stop<start):
    print('warning, your start and stop coordinates are inverted')
    return ''
  if(stop - start > 2000):
    print('warning, you asked for a seq larger than 2000 bp')
    return ''
  url_for_seq = 'http://genome.ucsc.edu/cgi-bin/das/hg38/dna?segment=chr'+str(chr_num_as_str)+':'+str(start)+','+str(stop)
  print(url_for_seq)
  df = pd.read_csv(url_for_seq)
  seq = ''
  index = 4
  while True:
    df.iat[index,0]
    seq = seq+df.iat[index,0]
    index = index +1
    if (df.iat[index,0]=="</DNA>"):
      print('break')
      break
  return seq

#chr17:7,676,521-7,676,622
#chr17:7,676,382-7,676,403
#chr17:7,675,993-7,676,272
exonA = getseqdatafromcoords('17', 7676521-20, 7676622+20)
exonA = exonA[0:20].lower()+exonA[20:len(exonA)-20].upper()+exonA[len(exonA)-20:].lower()
print(exonA)
exonB = getseqdatafromcoords('17', 7676382-20, 7676403+20)
exonB = exonB[0:20].lower()+exonB[20:len(exonB)-20].upper()+exonB[len(exonB)-20:].lower()
print(exonB)
exonC = getseqdatafromcoords('17', 7675993-20,7676272+20)
exonC = exonA[0:20].lower()+exonC[20:len(exonC)-20].upper()+exonC[len(exonC)-20:].lower()
print(exonC)


http://genome.ucsc.edu/cgi-bin/das/hg38/dna?segment=chr17:7676501,7676642
break
cttccaatggatccactcacAGTTTCCATAGGTCTGAAAATGTTTCCTGACTCAGAGGGGGCTCGACGCTAGGATCTGACTGCGGCTCCTCCATGGCAGTGACCCGGAAGGCAGTCTGGCTGctgcaagaggaaaagtgggg
http://genome.ucsc.edu/cgi-bin/das/hg38/dna?segment=chr17:7676362,7676423
break
agcccaacccttgtccttacCAGAACGTTGTTTTCAGGAAGTctgaaagacaagagcagaaa
http://genome.ucsc.edu/cgi-bin/das/hg38/dna?segment=chr17:7675973,7676292
break
cttccaatggatccactcacCCGTGCAAGTCACAGACTTGGCTGTCCCAGAATGCAAGAAGCCCAGACGGAAACCGTAGCTGCCCTGGTAGGTTTTCTGGGAAGGGACAGAAGATGACAGGGGCCAGGAGGGGGCTGGTGCAGGGGCCGCCGGTGTAGGAGCTGCTGGTGCAGGGGCCACGGGGGGAGCAGCCTCTGGCATTCTGGGAGCTTCATCTGGACCTGGGTCTTCAGTGAACCATTGTTCAATATCGTCCGGGGACAGCATCAAATCATCCATTGCTTGGGACGGCAAGGGGGActgtagatgggtgaaaagag


**Q3 Code (or pseudocode) for finding potential gRNAs**

In [ ]:
#See code above that is used by Q4

**Q4 Designing a gRNA library targeting coding sequence**


In [ ]:
from google.colab import files
#get sequence from github (input section 3.2)
#header, my_seq = get_github_fasta('https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/test_seq.fa')

#ALTERNATE - Get data from your PC - uncomment to try it out
#header, my_seq = get_fasta_from_pc()

#You can also put a sequence in here directly
#replace with imported sequence if your prefer, this sequence is for demo purposes
my_seq = 'AAAGGTTATGCAGCGCGTGAACATGATCATGGCAGAATCACCAGGCCTCATCACCATCTGCCTTTTAGGATATCTACTCAGTGCTGAATGTACAGGTTTGT'
#Dealing with sequence data (Section 3.3)
#You can change the protospacer sizeand PAM here
df = get_protospacer_list(20,'NGG',exonA)

#Output (Section 3.4)
df.to_csv('ExonA.csv')
files.download('ExonA.csv')

df = get_protospacer_list(20,'NGG',exonB)
df.to_csv('ExonB.csv')
files.download('ExonB.csv')
df = get_protospacer_list(20,'NGG',exonC)
df.to_csv('ExonC.csv')
files.download('ExonC.csv')



your sequence:
length: 142 bp
cttccaatggatccactcacAGTTTCCATAGGTCTGAAAATGTTTCCTGACTCAGAGGGGGCTCGACGCTAGGATCTGACTGCGGCTCCTCCATGGCAGTGACCCGGAAGGCAGTCTGGCTGctgcaagaggaaaagtgggg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

your sequence:
length: 62 bp
agcccaacccttgtccttacCAGAACGTTGTTTTCAGGAAGTctgaaagacaagagcagaaa


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

your sequence:
length: 320 bp
cttccaatggatccactcacCCGTGCAAGTCACAGACTTGGCTGTCCCAGAATGCAAGAAGCCCAGACGGAAACCGTAGCTGCCCTGGTAGGTTTTCTGGGAAGGGACAGAAGATGACAGGGGCCAGGAGGGGGCTGGTGCAGGGGCCGCCGGTGTAGGAGCTGCTGGTGCAGGGGCCACGGGGGGAGCAGCCTCTGGCATTCTGGGAGCTTCATCTGGACCTGGGTCTTCAGTGAACCATTGTTCAATATCGTCCGGGGACAGCATCAAATCATCCATTGCTTGGGACGGCAAGGGGGActgtagatgggtgaaaagag


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>